In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import statsmodels
import matplotlib.pyplot as plt
import requests
import json
from pandas import json_normalize
from IPython.display import HTML
%matplotlib inline

pd.options.display.precision = 15
pd.options.display.max_rows = 500

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import requests
import time, random

import re
import gensim
from gensim.models import CoherenceModel

C:\Users\Admin\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
#read in data
df1 = pd.read_excel('jobs_uae_AD.xlsx')
df2 = pd.read_excel('jobs_uae_dubai.xlsx')
frames = [df1,df2]
df = pd.concat(frames)
corpus = df['summary']
corpus.shape
str1 = listToString(corpus)

In [8]:
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()

#Function to pre-process the text information

def normalize_document(doc):
    
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A) #re.I (ignore case), re.A (ASCII-only matching)
    doc = doc.lower()
    doc = doc.strip()
   
    # tokenize document
    tokens = wpt.tokenize(doc)
    
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatise document from filtered tokens
    lem_text = [lemmatizer.lemmatize(i) for i in filtered_tokens]
    
    # Remove words that are only one character.
    lem_text = [token for token in lem_text if len(token) > 1]
    
    # Remove numbers, but not words that contain numbers.
    lem_text = [token for token in lem_text if not token.isnumeric()]
    
    doc = ' '.join(lem_text)
    return doc
str2 = normalize_document(str1)

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(corpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import gensim.corpora as corpora

#Tokenise the corpus
tokenized_corp = [word_tokenize(i) for i in norm_corpus]

# Create Dictionary
id2word = corpora.Dictionary(tokenized_corp)

#Remove words that don't feature 20 times and those that feature in over 50% of documents
id2word.filter_extremes(no_below=10, no_above=0.3)

texts = tokenized_corp

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=8, 
                                       random_state=123,
                                       chunksize=100,
                                       passes=500,
                                       per_word_topics=True)

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, n_jobs=1)
pyLDAvis.save_html(vis, 'LDA.html')
pyLDAvis.display(vis)

In [165]:
auth_endpoint = "https://auth.emsicloud.com/connect/token" # auth endpoint

client_id = "k3qyefj275x2i813" # replace 'your_client_id' with your client id from your api invite email
client_secret = "VtX86jAr" # replace 'your_client_secret' with your client secret from your api invite email
scope = "emsi_open" # ok to leave as is, this is the scope we will used

payload = "client_id=" + client_id + "&client_secret=" + client_secret + "&grant_type=client_credentials&scope=" + scope # set credentials and scope
headers = {'content-type': 'application/x-www-form-urlencoded'} # headers for the response
access_token = json.loads((requests.request("POST", auth_endpoint, data=payload, headers=headers)).text)['access_token'] # grabs request's text and loads as JSON, then pulls the access token from that


In [92]:
def extract_skills_list():
  all_skills_endpoint = "https://emsiservices.com/skills/versions/latest/skills" # List of all skills endpoint
  auth = "Authorization: Bearer " + access_token # Auth string including access token from above
  headers = {'authorization': auth} # headers
  response = requests.request("GET", all_skills_endpoint, headers=headers) # response
  response = response.json()['data'] # the data

  all_skills_df = pd.DataFrame(json_normalize(response)); # Where response is a JSON object drilled down to the level of 'data' key
  return all_skills_df

extract_skills_list()

,id,infoUrl,name,type.id,type.name
0,KS120P86XDXZJT3B7KVJ,https://skills.emsidata.com/skills/KS120P86XDX...,(American Society For Quality) ASQ Certified,ST3,Certification
1,KS126XS6CQCFGC3NG79X,https://skills.emsidata.com/skills/KS126XS6CQC...,.NET Assemblies,ST1,Hard Skill
2,KS1200B62W5ZF38RJ7TD,https://skills.emsidata.com/skills/KS1200B62W5...,.NET Framework,ST1,Hard Skill
3,KS126XW78QJCF4TRV2X7,https://skills.emsidata.com/skills/KS126XW78QJ...,.NET Framework 1,ST1,Hard Skill
4,KS126XY68BNKXSBSLPYS,https://skills.emsidata.com/skills/KS126XY68BN...,.NET Framework 3,ST1,Hard Skill
...,...,...,...,...,...
30503,ESCD425835A01B0C6EE7,https://skills.emsidata.com/skills/ESCD425835A...,pH Meters,ST1,Hard Skill
30504,KSQ1NQ6HNQ0PZ8J8FY0R,https://skills.emsidata.com/skills/KSQ1NQ6HNQ0...,phpBB,ST1,Hard Skill
30505,ESB5778D0DE341B0FBD2,https://skills.emsidata.com/skills/ESB5778D0DE...,qTest,ST1,Hard Skill
30506,KS440T963CCSV4W7BWJ5,https://skills.emsidata.com/skills/KS440T963CC...,spatialNET,ST1,Hard Skill


In [102]:
# Extract skills from a document
def extract_skills_from_document():
  skills_from_doc_endpoint = "https://emsiservices.com/skills/versions/latest/extract"
  text = "willing work hour per weekable join april may driver license willing take driving licenseup aed per monthpreferred healthcare others airline hospitality industry sector within uae onlyreporting operation manager senior operationswe currently working alongside partner united arab emiratesassisting search forgeneral practitioner general physician gpscommunicate using appropriate terminology guest family healthcare professionalsprovides expert appropriate medical service andhealthcare experience mustwe looking storekeeper medical center abu dhabiup aed per monthhealthcare assistant abu dhabi function performto provide range healthcare nonnursing duty ensuring efficient effective service tothe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualthe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualis responsible financialassessments generating report establishing company budget conduct risk assessment business initiativesas part delivery hero network talabat pioneer online food ordering industry middle eastidentify solve internal system issuescandidates call centre experience healthcare industryemirates hospital group looking experienced call centre executive join ourthe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualwe develop breakthrough therapy innovative healthcare solution area unmet medical need human animalsfluent arabic englishdermatologist start gp derma experience start speech therapist start arabcommunicates using appropriate terminology guest family healthcare professionalsreporting deputy medical director willhousecall product droncall primary healthcare provider healthtech startup implementing progressive solution community healthcarehigh tech healthcare art education sport government telecom retail printing design entertainment travel tourism hospitality real estatethe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualphysicians office hospital healthcare facilitiespatient aid responsible clinical task well administrative aswe develop breakthrough therapy innovative healthcare solution area unmet medical need human animalsa government healthcare facility abu dhabi looking temporary medical coder month subject extensionupdated new change healthcare rule regulationsan authorization clerk requested work private dental centre abu dhabithe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annual arabic customer service representative female urgentwith experience call centercan speak arabic englishcancelled visa visit visawe need arabic marketing executive year experience healthcare facility abu dhabi plastic shield work stationswith work experience medical center clinicwith good communication skill open nationalitiescancelled visa visit visaprovides pharmacological information answering question request healthcare professionalswe looking female pharmacistanaesthesia consultant actively participate teaching training junior anesthetist allied healthcare professionalsfeb amdiscuss patient goalpost physician surgeon healthcare teamcollaborate patient doctor referral note medical historyminimum year working experiencemust haad licenseeligibility lettermust bachelor degree nursingup aed monthshall develop implement review update comprehensive set quality safety policy procedure per doh jciahealthcare background mustblack pearl currently growing team consultantsthis role project abu dhabi bethe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualwe encourage explore different content area geography yearlong rotation one practice eg corporate finance socialplanning monitoring preparing presenting action plan respect several healthcare wide initiative presently undertakenthe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annual subsidized healthcare employee sponsored family member college communication medium science seek incumbent position year experience healthcare setting preferably within uae within outpatient outpatient large scale center cc cc cca cpc cpc nmc healthcare largest private healthcare company uae rank amongst leading fertility service provider world engineering degree tier university mba specialization operation banking energy material plus minimum year experience working uae specifically healthcare sector candidate must thorough understanding uae healthcare landscape previous healthcare operation healthcare insurance experience client leading hospital group uae requires headquarter abu dhabi minimum three year outpatient coding experience setting  hospital clinic related healthcare field required nmc healthcare largest private healthcare company uae rank amongst leading fertility service provider world qualified candidate posse university level degree and year healthcare work experience demonstrated sale marketing expertise sale university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual capital market private equity real estate hospitality healthcare retail ensure international best finance accounting practice client offer service include limited healthcare food nutrition integrated facility management delegate task nurse healthcare assistant general practitioner healthcare assistant thing like draw blood take height weight project manager lead implementation support team planning executing monitoring controlling closing ai healthcare company university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual iqarus proud history extending year delivering fully integrated healthcare solution complex demanding operating environment relevant qualification beauty therapy active license practice beauty therapy proven experience working beauty industry ensure efficient effective communication patient family member healthcare team gp surgery gp pediatrics offer endeavor provide service friendly efficient professional environment delivering premium healthcare solution adhere following prescribed healthcare plan may include assisting exercise administering medication high school diploma required well established healthcare chain looking candidate responsibility include monitoring patient condition assessing need selling company medical product hospital clinic pharmacy relevant healthcare organization university benefit package highly attractive competitive salary housing furniture allowance annual vacation airline ticket least year experience healthcare setting client leading hospital group uae requires headquarter abu dhabi university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual healthcare assistant urgently needed applicant start immediately prioritized preferrably vast knowledge experience assisting process manage facility healthcare professional licensure haad moh dha process data flow awareness hr policy procedure zayed university benefit package highly attractive competitive salary free tax united arab emirate housing furniture allowance  university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual seeking highly skilled gp doctor dermatology aesthetic experience join growing healthcare team social distancing guideline place aware current future trend healthcare technology healthcare funding mechanism ge healthcare equal opportunity employer inclusion via medica international healthcare continuously looking male female nurse assistant part home care division team university benefit package highly attractive competitive salary housing furniture allowance annual vacation airline ticket endeavor provide service friendly efficient professional environment delivering premium healthcare solution adhere minimum year working experience related field must haad license eligibility letter must bachelor degree nursing multitasking listening verbal communication education experience licensing requirement registered nurse rn haad license vamed may refer record international success healthcare area prevention acute care rehabilitation nursing prescription processing dispensing per doh law rule regulation collaborate physician discus potential issue patient prescription dermatologist overall responsible financial success metric dermat service delivering service excellence alongside great customer provision healthcare within scope action highest quality recruiting part time dentist periodontist one favorite client analysis verifies ensures accuracy data sent healthcare service including editing appropriate sonographer abu dhabi function perform sama alshamkha medical center brings quality healthcare service closer patient family design fabricate use orthodontic appliance communicate using appropriate terminology guest family healthcare professional provides expert appropriate medical service subsidized healthcare employee sponsored family member zayed university leading university united arab emirate innovative kare plus mena market leading permanent staffing service provider dedicated sourcing niche skilled high caliber talent doctor nurse  university benefit package highly attractive withcompetitive salary free tax  cash housing furnitureallowance annual vacation dedicated improving healthcare access everyone leaving world better found instead job mission year total experience air conditioning work year experience healthcare facility maintenance must vacancy multi specialty medical center jci accreditation town center al preference given candidate based al segment focus hotel real estate healthcare digital building key account manager primary objective role lead develop  provides direct healthcare student staf develops individualized healthcare plan refers follow child chronic condition successful applicant work one two collaborative project supporting decision maker variety domain healthcare epidemiology  university benefit package includes salary free income tax uae housing allowance furniture allowance annual vacation airline ticket university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual vamed may refer record international success healthcare area prevention acute care rehabilitation nursing university benefit package highly attractive competitive salary free tax united arab emirate housing furniture circinus llc seeking mid career intelligence analyst address client open source intelligence osint need middle east based healthcare management consultancy looking sale  previous experience healthcare sector would advantage alpha al urgently looking assistant nurse join company following qualification bachelor science nursing first class healthcare education superior living accommodation cultural attraction find ideal career global university benefit package highly attractive competitive salary free tax uae housing furniture allowance annual vacation experience healthcare industry preferred experience hl fast healthcare interoperability resource fhir standard team people responsible delivering essential public service around world area including defence transport justice immigration currently looking additional registered male nurse work site clinic camp currently staying abu dhabi aed per month vamed may refer record international success healthcare area prevention acute care rehabilitation nursing via medica international healthcare continuously looking male female nurse assistant part home care division team via medica international healthcare llc urgently looking fork male registered nurse part academic health division male site nurse registered nurse practical nurse doh license  eligibility letter dha preferred  inhealth connect stakeholder within healthcare community providing user friendly interface facilitates easy communication across assessing implementing planning evaluating patient nursing care plan working healthcare team member eligibility letter dha preferred  inhealth connect stakeholder within healthcare community providing user friendly interface facilitates easy communication across two year experience healthcare domain understand current data architecture expand optimize company data pipeline architecture well primarily responsible consulting providing clinical technical information healthcare professional within assigned territory university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual zayed university benefit package highly attractive competitive salary free tax united arab emirate housing furniture allowance  university benefit package highly attractive competitive salary free tax united arab emirate housing furniture allowance  within healthcare environment excellent communication skill written verbal education requirement ba bsc strong computer skill experience healthcare database application experienced city abu dhabi entertain knowledge healthcare health position research project mobile healthcare working experience within healthcare supply chain strong working knowledge principal distributor experience warehouse management inter professional healthcare quality improvement collaborates healthcare consumer family others conduct nursing practice conduct method evaluation development per standard policy procedure complying safety health quality control program procedure strategy analyst support critical strategy development project healthcare across diagnostics digital health bio pharmaceutical segment providing drug information healthcare professional patient client leading hospital group uae requires headquarter abu dhabi inter professional healthcare quality improvement collaborates healthcare consumer family others conduct nursing practice collaborating dental healthcare specialist develop suitable treatment plan patient specialist prosthodontist doctor see nmc healthcare largest private healthcare company uae rank amongst leading fertility service provider world experience healthcare advantage strong analytical skill ability perform predictive modeling quantitative qualitative based data communicate using appropriate terminology guest family healthcare professional managing directing daily operation pharmacy understanding healthcare licensing title criterion healthcare management consultancy located al suitable candidate must working experience healthcare facility uae looking patient nurse following qualification required arabic occupational therapist major healthcare group uae valid haad dha moh license year experience occupational therapist vamed may refer record international success healthcare area prevention acute care rehabilitation nursing master degree information technology computer science social science education mathematics statistic healthcare administration related field annual flight allowance accommodation provided world class medical healthcare benefit subject status  repton school abu dhabi currently looking doh registered passer male nurse work site nurse must doh haad licensed passer previous working experience healthcare setting government health service would advantage job description research assistant position must bachelor degree must haad dha moh license rn dental assistant  must dental training certification ass chart observation patient condition visit complete evaluation task including reviewing medication vital sign center support environment best practice utilizing effective triage system prioritize client care need exceptional facility design advising customer selection medication brand medical equipment healthcare supply working healthcare professional plan monitor leading cancer treatment team various healthcare professional looking caring dedicated oncologist join team diagnosing treating  female doh passer flexible hardworking job type full time permanent work remotely  inter professional healthcare quality improvement collaborates healthcare consumer family others conduct nursing practice nmc healthcare largest private healthcare company uae rank amongst leading fertility service provider world housecall product droncall primary healthcare provider healthtech startup implementing progressive solution community healthcare assessing implementing planning evaluating patient nursing care plan working healthcare team member aed per month good experience healthcare domain advantage looking microsoft d functional report consultant good experience retail endeavor provide service friendly efficient professional environment delivering premium healthcare solution adhere minimum year working experience must haad license eligibility letter must bachelor degree nursing aed per month university benefit package highly attractive competitive salary free tax  housing allowance furniture allowance annual well reputed healthcare provider based heart abu dhabi part expansion service currently hiring registered nurse via medica international healthcare llc continuously looking registered nurse part home care division abu dhabi looking specialist doctor see working living united arab emirate salary permanent consultant specialist jcia private healthcare group one largest uae employ personnel hospital medical center across uae name wonderful opportunity available experienced occupational therapist abu dhabi uae currently looking occupational therapist via medica international healthcare llc continuously looking female registered nurse part academic health division research existing technology across global healthcare company liaise team gather requirement align overall strategy  registered nurse require no leading healthcare group uae  valid uae license dha haad moh  location abu dhabi al responsible day day operation including regulatory compliance supervise receiving warehousing picking packing dispatch order meet one leading healthcare group uae looking passionate specialist critical care medicine abu dhabi uae location attractive salary wonderful opportunity available experienced pharmacy manager abu dhabi uae currently looking pharmacy mangaer extremely well client middle east leading provider specialised long term care rehabilitation home healthcare service region pioneer experience healthcare sector knowledge healthcare data system clinical information system still hiring additional male site nurse active doh rn license doh passer currently staying abu dhabi university benefit package includes salary free income tax uae housingallowance furniture allowance annual vacation airline ticket immediate start icu consultant vacancy uae month contract  prospect health recruiting icu consultant work prestigious government via medica international healthcare llc continuously looking male registered nurse part home care division abu dhabi ability establish maintain good rapport child staff parent responsible maintain child medical record given medical care must minimum year working experience healthcare facility monitor ass patient condition preparation administration medication  client well known healthcare facility uae currently recruiting physiotherapist experience patient learning disability university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual vamed may refer record international success healthcare area prevention acute care rehabilitation nursing successful candidate must research interest general area informatics cybersecurity focus information system security iot  sama alshamkha medical center brings quality healthcare service closer patient family greet client come procedure  search hijama cupping therapist naturopathic treatment division healthcare service based abu dhabi immediate start icu consultant vacancy uae month contract  prospect health recruiting icu consultant work prestigious government davidson management consultant mandated recruit native arabic speaking consultant dermatologist client multinational family medicine specialist responsible total management patient supervision senior doctor collaboration relevant qualification beauty therapy active license practice beauty therapy proven experience working beauty industry paid international healthcare plan family circinus llc seeking senior intelligence analyst sia  team lead support client client well known healthcare facility based uae looking homecare team leader home healthcare unit based abu dhabi al vamed may refer record international success healthcare area prevention acute care rehabilitation nursing seeking director clinical operation nursing background vast experience within diabetes care highly established hospital group dietitian al united arab emirate currently looking dietitian well established hospital group united arab emirate role compliant state healthcare regulatory requirement conducting physical psychological assessment client developing following treatment collaborating doctor healthcare professional develop improved diet healthcare plan patient hospital known high quality healthcare service also commitment medical welfare society large client well known healthcare facility uae currently recruiting position tissue viability nurse based abu dhabi currently opening specialist emergency medicine profile one leading healthcare group uae  reputed healthcare group serving region year hospital abu dhabi dubai mandated davidson management zayed university benefit package highly attractive competitive salary free tax united arab emirate housing furniture allowance  currently looking paediatric physiotherapist new paediatric facility al united arab emirate client focus rehabilitation  job description position research project mobile healthcare minimum qualification master degree research training proficiency wonderful opportunity available experienced palliative care consultant abu dhabi currently looking palliative care consultant iqarus proud history extending year delivering fully integrated healthcare solution complex demanding operating environment iqarus proud history extending year delivering fully integrated healthcare solution complex demanding operating environment provide healthcare student staff coordinate referral private healthcare provider serve liaison school personnel family one biggest healthcare group want hire general critical nurse hospital homecare nurse job abu dhabi hour work hour day week  employing thousand healthcare professional within uae private group hospital pioneer cutting edge healthcare service region minimum year total experience uae year experience healthcare facility maintenance must one pioneer healthcare sector abu dhabi hospital abu dhabi made name region leader use cutting edge hiring company specialist home healthcare highly trained nurse care patient dubai abu dhabi abu dhabi uae client university benefit package highly attractive competitive salary housing furniture allowance annual vacation airline ticket client well known healthcare facility based uae looking rehabilitation therapy leader unit based abu dhabi registration pharmacy technician relevant healthcare authority desired least year experience pharmaceutical field within client middle east leading provider specialised long term care rehabilitation home healthcare service region pioneer outstanding opportunity available number experienced icu nurse based united arab emirate  preferred healthcare others airline hospitality  industry sector within uae reporting operation manager senior operation outstanding opportunity available number experienced paediatric nurse united arab emirate hospital offer following benefit outstanding opportunity available paediatric clinical nurse educator united arab emirate hospital offer following benefit joining organisation grown significantly recent year become leading provider inpatient rehabilitation long term care  assessing implementing planning evaluating patient nursing care plan working healthcare team member continuing initiative engage best healthcare professional uae across globe opened hiring process currently opening specialist dermatologist female profile one leading healthcare group uae  wonderful opportunity available experienced speech language therapist uae currently looking speech language therapist least year hospital experience year old fit work  writing component oet passer least year hospital experience year old fit work  writing component oet passer rcsi exam paid employer  one biggest healthcare group want hire general critical nurse hospital homecare nurse job abu dhabi hour work hour day week  least year consultant level experience tier country specified uae healthcare regulator consultant internal medicine gastroenterology vacancy multi specialty medical center jci accreditation town center al candidate haad license doh license wonderful opportunity available experienced consultant paediatric neurologist abu dhabi currently looking consultant paediatric leading healthcare facility abu dhabi urgently looking recruit rn nurse female candidate haad doh license uae experience eligible responsible orientation induction training new junior registered nurse practical nurse healthcare assistant minimum year working experience must haad license eligibility letter must bachelor degree nursing aed per month university benefit package highly attractive competitive salary free tax uae housing furniture allowance annual vacation wonderful opportunity available experienced paediatric charge nurse manager uae currently looking paediatric charge nurse  good working exp india reputed healthcare group currently one leading healthcare group uae looking passionate leading healthcare management company based abu dhabi looking forward recruit nurse abu dhabi location client healthcare innovation research company currently covid trial vaccination genomic sequencing research joining organisation grown significantly recent year become leading provider inpatient rehabilitation long term care  maintain safe clean working environment compliance healthcare procedure regulation qualified laser technician university benefit package highly attractive competitive salary free tax uae housing furniture allowance annual vacation inter professional healthcare quality improvement collaborates healthcare consumer family others conduct nursing practice year omni international consultant supplied high caliber personnel global construction aviation banking logistics hospitality  davidson management consultant mandated recruit native arabic speaking consultant ent client multinational european university benefit package includes salary free income tax uae housing allowance furniture allowance annual vacation airline ticket client well known healthcare facility based abu dhabi currently recruiting senior staff nurse experience patient one biggest healthcare group want hire general critical nurse hospital hospital nurse job abu dhabi hour work hour day week  looking new position united arab emirate medacs healthcare seeking western trained midwife private hospital al al university benefit package highly attractive competitive salary united arab emirate housing furniture allowance annual vacation outstanding opportunity available experienced tissue viability wound care nurse specialistin united arab emirate wonderful opportunity available cerebral palsy nurse specialist currently looking cerebral palsy nurse specialist extremely well wonderful opportunity available acute medical senior staff nurse currently looking acute medical senior staff nurse extremely well minimum year working experience must haad license eligibility letter must bachelor degree nursing aed month davidson management consultant mandated recruit native arabic speaking female consultant pediatrician client multinational client well known healthcare facility based abu dhabi currently recruiting staff nurse experience patient intellectual wonderful opportunity available experienced neurological physiotherapist abu dhabi uae currently looking physiotherapist wonderful opportunity available experienced rehabilitation physician abu dhabi currently looking rehabilitation physician registered homecare nurse haad doh one largest healthcare facility abu dhabi hiring client male female apply client middle east leading provider specialised long term care rehabilitation home healthcare service region pioneer university benefit package highly attractive competitive salary united arab emirate housing furniture allowance annual vacation client well known healthcare facility uae currently recruiting senior occupational therapist experience dealing patient client well known healthcare facility uae currently recruiting occupational therapist neuro rehab experience dealing one client healthcare service provider urgently looking hire nurse center dubai abudhabi job posted jan client well known healthcare facility uae currently recruiting occupational therapist outpatient experience dealing minimum year working experience must haad license eligibility letter must bachelor degree nursing aed month university benefit package includes salary free income tax uae housingallowance furniture allowance annual vacation airline ticket wonderful opportunity available experienced neurological occupational therapist abu dhabi uae continued care program capable caring wonderful opportunity available experienced occupational therapist uae currently looking occupational therapist join highly minimum year working experience must haad license eligibility letter must bachelor degree nursing aed month client well known healthcare facility uae currently recruiting occupational therapist extensive experience patient wonderful opportunity available palliative senior staff nurse currently looking palliative senior staff nurse extremely well immediate start icu consultant vacancy uae month contract  prospect health recruiting icu consultant work prestigious government seeking diabetes nurse educator well esteemed hospital group united arab emirate abu dhabi role working within group seeking director clinical operation nursing background vast experience within high dependency critical care highly established client well known healthcare facility based abu dhabi currently recruiting arabic staff nurse experience patient client middle east leading provider specialised long term care rehabilitation home healthcare service region pioneer wonderful opportunity available experienced physical medicine rehabilitation physician abu dhabi hospital offer following benefit outstanding opportunity available senior dialysis nurse icu experience united arab emirate hospital offer following benefit outstanding opportunity available number experienced senior charge icu nurse based united arab emirate minimum year working experience must haad license eligibility letter must bachelor degree nursing aed month  immediate start icu consultant vacancy uae month contract  prospect health recruiting icu consultant work prestigious government amana healthcare part mubadala healthcare network mubadala investment company sovereign investor  practice medicine way meant amana healthcare part mubadala healthcare network mubadala investment company sovereign investor  practice medicine way meant vacancy multi specialty medical center jci accreditation town center al candidate haad license doh license willing work home healthcare nurse client one leading company uae licensed certified field medical service seeking senior staff nurse strong background experience working within diabetes well esteemed hospital group united arab wonderful opportunity available respiratory cardiac nurse specialist currently looking respiratory cardiac nurse specialist extremely looking detail oriented general accountant setting perform coordinate accounting duty within organization experience healthcare sector advantage must holder bachelor degree accounting finance business administration master mba greet visitor patient determine need guide accordingly answer query provide information directly person amana healthcare part mubadala healthcare network mubadala investment company sovereign investor  practice medicine way meant maintain safe clean working environment compliance healthcare procedure regulation effect comprehensive nursing care plan patient willing work hour per week able join april may driver license willing take driving license aed per month  preferred healthcare others airline hospitality  industry sector within uae reporting operation manager senior operation collaborating doctor healthcare professional develop improved diet healthcare plan patient currently working alongside partner united arab emiratesassisting search forgeneral practitioner general physician gp  housecall product droncall primary healthcare provider healthtech startup implementing progressive solution community healthcare communicates using appropriate terminology guest family healthcare professional reporting deputy medical director consultant paediatric pulmonology sleep medicine abu dhabi currently looking consultant pulmonologist sleep medicine well established minimum year working experience must haad license eligibility letter must bachelor degree nursing aed month communicates using appropriate terminology guest family healthcare professional reporting deputy medical director communicate using appropriate terminology guest family healthcare professional provides expert appropriate medical service collaborating doctor healthcare professional develop improved diet healthcare plan patient  preferred healthcare others airline hospitality  industry sector within uae reporting operation manager senior operation willing work hour per week able join april may driver license willing take driving license aed per month housecall product droncall primary healthcare provider healthtech startup implementing progressive solution community healthcare seeking senior nurse manager director strong background experience working within diabetes well esteemed hospital group united immediate start icu consultant vacancy uae month contract  prospect health recruiting icu consultant work prestigious government  preferred healthcare others airline hospitality  industry sector within uae reporting operation manager senior operation housecall product droncall primary healthcare provider healthtech startup implementing progressive solution community healthcare willing work hour per week able join april may driver license willing take driving license aed per month communicates using appropriate terminology guest family healthcare professional reporting deputy medical director collaborating doctor healthcare professional develop improved diet healthcare plan patient customer centric service oriented minimum year healthcare experience customer care customer service front office general manager responsible leading development execution organization long term strategy view creating shareholder humble family business started bank dubai creek s al futtaim expanded presence country portfolio demonstrates understanding home healthcare industry refers specific medical query issue healthcare professional accordance soukare com dubai based online healthcare start aiming define healthcare convenience region provide minute delivery service comprehensive healthcare well subsidized gym membership parking part delivery hero network talabat pioneer online food minimum year experience receptionist reputed healthcare facility looking arabic female receptionist branch dubai based dubai deliver unique value proposition accelerates growth penetration wallet share maintenance contract project proficient knowledge country healthcare insurance market proficient knowledge gcc country healthcare provider trend behavior responsible performing provider audit campaign identifying unfavorable trend provider billing maximize ensure compliance legal capital college invite application healthcare management expert innovation danish healthcare system national local experience coordinate physician accurate clinical documentation avoid insurance rejection minimize denial reviewing patient bill accuracy year experience assistant healthcare pharmaceutical industry knowledge software hardware function part delivery hero network talabat pioneer online food ordering industry middle east daily coordination store picker payroll service provider recruitment consultancy learning development service provider remuneration benefit service provider healthcare university benefit package highly attractive competitive salary cash housing furniture allowance annual vacation airline ticket minimum year sale experience related healthcare experience timing follow order value proposition order decision maker university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual iqarus proud history extending year delivering fully integrated healthcare solution complex demanding operating environment ideal candidate based dubai year sale experience healthcare pharma related business nov middle east fza uae prestigious healthcare group uae looking accountant uae experience salary given time day leave yearly air ticket allowance allocation selected office based duty operation coordinator work within international division viamedica international healthcare minimum year healthcare experience currently seeking experienced senior store assistant succeed role expected promote solution real impact life across planet enabling industry improving healthcare enhancing security advancing connectivity supporting adopt proactive approach recognizing quality system improvement opportunity deficiency context emerging regulation change year uae healthcare industry advanced excel skill including vlookups pivot table experience general ledger function previous experience healthcare industry plus dubai based clinic look female customer service exceutive business development department candidate experience healthcare industry emirate hospital group looking experienced data analyst join team minimum year experience healthcare industry pharmacy industry preferred social distancing guideline place apex clinic group ambitious fast growing entity field healthcare looking enthusiastic smart business analyst minimum year work experience healthcare insurance specifically claim approval network provider relationship university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual university benefit package highly attractive competitive salary free tax  cash housing annual vacation airline ticket looking compassionate oral dental surgeon specialist consultant provide long term comprehensive care patient manage common humble family business started bank dubai creek s al futtaim expanded presence country portfolio opportunity work world leading organization healthcare finance telecommunication industry knowledgeable medical terminology appropriate level healthcare healthcare delivery system maintain develop provide specialist advice people shaping world life work play next generation advancement healthcare life science electronics looking driver home care division valid uae driving license preferred  assisting nursing team providing nursing care assist activity daily living getting patient ready transport department effort drive academic advancement extend quality medical service support local healthcare industry receive cv time apply position application cancelled within hour detail deleted knowledge understanding environmental health policy including limited ge healthcare ehs policy ge healthcare fleet rule etc  minimum year experience healthcare industry good knowledge insurance submission resubmission follow good knowledge excel healthcare experience involvement plus urgently looking product specialist megamind solution uae minimum year experience nurse dental assistant professional level experience provide visa dhcc free zone license sponsoring good salary package travel ticket  worked dubai minimum year healthcare first response healthcare frh dubai leading doctor call service provider proudly first response healthcare llc jci accredited home healthcare organization one dubai leading home healthcare company university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual bmc luxury medical center specializing aesthetic hair transplant looking nurse meet criterias updated new change healthcare rule regulation must experience field knowledgeable insurance company approval  good working experience dubai reputed healthcare group looking passionate dermatologist dubai uae location training healthcare assistant include basic caring skill work towards care certificate caregiver  university benefit package highly attractive competitive salary free tax  housing furniture allowance annual vacation company deep technical clinical analytical expertise work healthcare provider across india top generous travel benefit including discounted flight hotel stay around world managerial role also excellent leave looking marketing sale manager healthcare experience medical sale field especially dialysis machine renal product soukare com dubai based online healthcare start aiming define healthcare convenience region provide minute delivery service company grown exponentially investment number business across sector real estate manufacturing industrial healthcare education premium clinic located dubai healthcare city looking insurance coordinator medical secretary social distancing guideline place company grown exponentially investment number business across sector real estate manufacturing industrial healthcare education strong technical knowledge regarding healthcare product  year experience customer service healthcare industry  role based dubai healthcare strategy team offer strategic advice broad range stakeholder healthcare operator pharmaceutical life science organization  humble family business started bank dubai creek s al futtaim expanded presence country portfolio updated new change healthcare rule regulation must experience field knowledgeable insurance company approval  meraas dubai based conglomerate aim enhance emirate cityscape human centric approach developing asset real estate tourism university benefit package highly attractive competitive salary free tax  cash housing furniture allowance annual awareness current future trend healthcare technology healthcare funding mechanism role accountable grow sale revenue margin provide quality service treatment patient present clinic development healthcare service dubai looking light vehicle driver healthcare group dubai le age proffered valid uae driving license preferred  currently seeking highly skilled compassionate registered nurse succeed role proven skill committed first response healthcare llc one dubai leading home healthcare company highly trained qualified staff doctor specialize city doctor healthcare urgently hiring nurse assistant join home care division team nurse assistant providing treatment patient ignite search selectionemployment type help company individual protect ipr uae qatar oman bahrain kuwait office client leader innovator dha licensed nurse assistant nurse apply year experience preferred fresh holding dha licens welcome well top generous travel benefit including discounted flight hotel stay around world managerial role also excellent leave meraas dubai based conglomerate aim enhance emirate cityscape human centric approach developing asset real estate tourism play increasingly vital role region advising leading private public institution core sector including real estate development retail team people responsible"
  payload = "{ \"text\": \"... " + text + " ...\", \"confidenceThreshold\": " + "0.4" + " }"

  headers = {
      'authorization': "Bearer " + access_token,
      'content-type': "application/json"
      }

  response = requests.request("POST", skills_from_doc_endpoint, data=payload.encode('utf-8'), headers=headers)

  skills_found_in_document_df = pd.DataFrame(json_normalize(response.json()['data'])); # Where response is a JSON object drilled down to the level of 'data' key
                                            
  return skills_found_in_document_df
  
extract_skills_from_document()

,confidence,skill.id,skill.infoUrl,skill.name,skill.tags,skill.type.id,skill.type.name
0,1.000000000000000,KS12027766237GNG6G37,https://skills.emsidata.com/skills/KS120277662...,Quality Management Systems,"[{'key': 'wikipediaExtract', 'value': 'A quali...",ST1,Hard Skill
1,1.000000000000000,KSQPGV2XZO8UMNCC49LK,https://skills.emsidata.com/skills/KSQPGV2XZO8...,Detail Oriented,[],ST2,Soft Skill
2,1.000000000000000,KS122556LMQ829GZCCRV,https://skills.emsidata.com/skills/KS122556LMQ...,Communications,"[{'key': 'wikipediaExtract', 'value': 'Communi...",ST2,Soft Skill
3,1.000000000000000,KS123QD780L8N96VD1RM,https://skills.emsidata.com/skills/KS123QD780L...,Excel Pivot Tables And Charts,[],ST1,Hard Skill
4,1.000000000000000,KS124SJ67KB5YN7RVS52,https://skills.emsidata.com/skills/KS124SJ67KB...,House Call,"[{'key': 'wikipediaExtract', 'value': 'A house...",ST1,Hard Skill
5,1.000000000000000,KS126C26VWW3SXG5RHX9,https://skills.emsidata.com/skills/KS126C26VWW...,Medication Administration,"[{'key': 'wikipediaExtract', 'value': 'A Medic...",ST1,Hard Skill
6,1.000000000000000,KS124726L50CX4ZNHYK2,https://skills.emsidata.com/skills/KS124726L50...,Gastroenterology,"[{'key': 'wikipediaExtract', 'value': 'Gastroe...",ST1,Hard Skill
7,1.000000000000000,KS125S5622QQY5R0VFZB,https://skills.emsidata.com/skills/KS125S5622Q...,Learning Disabilities,"[{'key': 'wikipediaExtract', 'value': 'Learnin...",ST1,Hard Skill
8,1.000000000000000,KS128006L3V0HM2B26N5,https://skills.emsidata.com/skills/KS128006L3V...,Predictive Modeling,"[{'key': 'wikipediaExtract', 'value': 'Predict...",ST1,Hard Skill
9,1.000000000000000,KS1227V6WBR3BH3SJYSZ,https://skills.emsidata.com/skills/KS1227V6WBR...,Information Technology,"[{'key': 'wikipediaExtract', 'value': 'Informa...",ST2,Soft Skill


In [6]:
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1

In [87]:

str2

'willing work hour per weekable join april may driver license willing take driving licenseup aed per monthpreferred healthcare others airline hospitality industry sector within uae onlyreporting operation manager senior operationswe currently working alongside partner united arab emiratesassisting search forgeneral practitioner general physician gpscommunicate using appropriate terminology guest family healthcare professionalsprovides expert appropriate medical service andhealthcare experience mustwe looking storekeeper medical center abu dhabiup aed per monthhealthcare assistant abu dhabi function performto provide range healthcare nonnursing duty ensuring efficient effective service tothe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualthe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualis responsible financialassessments generating report establishi

In [39]:
text = str1


In [88]:
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
answer = ''.join(filter(whitelist.__contains__, str2))

In [89]:
answer


'willing work hour per weekable join april may driver license willing take driving licenseup aed per monthpreferred healthcare others airline hospitality industry sector within uae onlyreporting operation manager senior operationswe currently working alongside partner united arab emiratesassisting search forgeneral practitioner general physician gpscommunicate using appropriate terminology guest family healthcare professionalsprovides expert appropriate medical service andhealthcare experience mustwe looking storekeeper medical center abu dhabiup aed per monthhealthcare assistant abu dhabi function performto provide range healthcare nonnursing duty ensuring efficient effective service tothe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualthe university benefit package highly attractive competitive salary free tax uae cash housing furniture allowance annualis responsible financialassessments generating report establishi